In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
from os import fdopen, remove, walk
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

In [4]:
DATASET_PATH = "/Users/noopy/covid19_unknown_spread/dataset"
datasets = glob.glob(f"{DATASET_PATH}/*.csv")
datasets

['/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_31_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_30_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_17_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_21_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_7_04_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_20_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_16_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_7_09_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_9_01_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_15_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_9_02_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_19_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_23_.csv',
 '/Users/noopy/covid19_unknown_spread/dataset/seoul_covid_8_22_.csv',
 '/Users/noopy/covid

In [4]:
df_temp = pd.read_csv('./dataset/seoul_covid_9_04_.csv', encoding="utf-8")

In [5]:
df_temp.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
2759,1492,13805,7.20.,은평구,-,종로구 지인모임 신명투자,퇴원
2512,1739,14789,8.13.,성북구,-,고양시 반석교회 관련 (케네디상가),퇴원
1597,2654,16923,8.21.,광진구,-,순복음 강북교회,NaN
472,3779,19768,8.30.,도봉구,-,확인 중,NaN
2411,1840,15015,8.14.,송파구,-,성북구 사랑제일교회 관련,퇴원
3184,1067,12002,6.11.,은평구,-,kb 생명보험 관련,퇴원
2181,2070,15463,8.16.,성북구,-,성북구 사랑제일교회 관련,NaN
2144,2107,15570,8.17.,강북구,-,성북구 사랑제일교회 관련,NaN
2191,2060,15388,8.16.,성북구,-,성북구 사랑제일교회 관련,NaN
1530,2721,16770,8.21.,성북구,-,타시도 확진자 접촉,NaN


In [6]:
# overwrite whatever cell value that contains "확인" as "확인 중"
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"] = "확인 중"

# check whether those two arrays are combined
df_temp.loc[df_temp["접촉력"].str.contains("확인"),"접촉력"].unique()

array(['확인 중'], dtype=object)

In [7]:
df_temp.sample(15)

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
4113,138,7683,3.9.,성남시,-,확인 중,퇴원
4013,238,8049,3.13.,동대문구,-,동대문구 관련,퇴원
728,3523,19025,8.27.,관악구,-,확인 중,NaN
2469,1782,14973,8.14.,노원구,-,골드트레인 관련,퇴원
1495,2756,17046,8.22.,강동구,-,골드트레인,NaN
3071,1180,12284,6.18.,양천구,-,확인 중,퇴원
1465,2786,17340,8.22.,강서구,-,광화문집회 관련,NaN
929,3322,18501,8.26.,기타,-,타시도 확진자 접촉,NaN
2153,2098,15575,8.17.,중랑구,-,성북구 사랑제일교회 관련,퇴원
2122,2129,15690,8.17.,동대문구,-,기타 확진자 접촉,퇴원


In [16]:
df_temp["접촉력"] = df_temp["접촉력"].str.replace("관련", "")
df_temp["접촉력"] = df_temp["접촉력"].str.strip()

In [18]:
infection_paths = df_temp["접촉력"].unique()
print(len(infection_paths))
infection_paths

223


array(['송파구 소재 병원', '노원구 빛가온교회', '강동구 소재 병원', '성북구 사랑제일교회', '8.15도심집회',
       '확인 중', '도봉구 운동시설', '타시도 확진자 접촉', '기타 확진자 접촉', '다래경매', '노원구 기도모임',
       '동작구 카드 발급업체', '광진구 소재병원', '성북구 체대입시', '성북구 요양시설', '중구 소재 은행',
       '중랑구 소재 체육시설', '관악구 판매업소', '동작구 소재 서울신학교', '용인시 우리제일교회',
       '노원구 손해보험', '구로구 보성운수', '강서구 서울대효요양병원', '서초구 장애인교육시설', '서대문구 지인모임',
       '강북구 일가족', '중구 하나은행본점', '강서구 보안회사', '롯데리아 종사자 모임', '해외 접촉 추정',
       '8.15도심집회(순복음 강북교회)', '영등포구 권능교회', '구로구 아파트', '강남구 소재 아파트',
       '제주 게스트하우스', '성북구 벧엘장로교회', '중앙보훈병원', '관악구 김혜근의원', '동대문구 sk탁구클럽',
       '동작구 스터디카페', '극단 산', 'KT가좌지사', '군인권센터', '동작구 요양시설', '여의도 순복음교회',
       '중구 보험회사(현대해상)', '강서구 병원', '종로구 혜화경찰서', '양천구 되새김교회', '현대커머셜',
       '8.15도심집회(녹색병원)', '영등포 IFC몰 오케스트로', '고양시 반석교회(케네디상가)', '롯데 자산개발',
       '골드트레인', '한양대병원', '은평구 헤어콕', '순복음 강북교회', '서대문구 지인 모임', '강동구 어린이집',
       '광화문집회', '은평구 성경공부모임', '관악구 요양병원', '고대 안암병원', '마포구 푸본생명콜센터',
       '성동구 가족', '양천구 되새김 교회', '중구 통일상가', '고양시 반석교회  (케네디상가)',
       '강남구 판매업소(

In [19]:
df_temp.to_csv("./dataset_predict/df_wrangle.csv", index=False)

In [28]:
df_date = df_temp.sort_values(["연번"], ascending=False)
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9.04.,타시,-,송파구 소재 병원,NaN
1,4250,20979,9.04.,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9.04.,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9.04.,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9.03.,성북구,-,8.15도심집회,NaN


In [29]:
df_date["확진일"] = df_date["확진일"].str.replace(".", "-")
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9-04-,타시,-,송파구 소재 병원,NaN
1,4250,20979,9-04-,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9-04-,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9-04-,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9-03-,성북구,-,8.15도심집회,NaN


In [30]:
df_date["확진일"] = df_date["확진일"].str[:-1]
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,9-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,9-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,9-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,9-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,9-03,성북구,-,8.15도심집회,NaN


In [31]:
df_date["확진일"] = "2020-0" + df_date["확진일"]
df_date.head()

In [33]:
df_date.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [34]:
df_date.to_csv("./dataset_predict/df_wrangle.csv", index=False)

In [35]:
df_date.tail()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
4246,5,9,2020-01-31,성북구,-,#5 접촉,퇴원
4247,4,7,2020-01-30,마포구,중국 우한시,해외 접촉,퇴원
4248,3,6,2020-01-30,종로구,-,#3 접촉,퇴원
4249,2,5,2020-01-30,중랑구,중국 우한시,해외 접촉,퇴원
4250,1,2,2020-01-24,강서구,중국 우한시,해외 접촉,퇴원


In [41]:
df = df_date.copy()
df.head()

,연번,환자,확진일,거주지,여행력,접촉력,퇴원현황
0,4251,미부여,2020-09-04,타시,-,송파구 소재 병원,NaN
1,4250,20979,2020-09-04,노원구,-,노원구 빛가온교회,NaN
2,4249,20974,2020-09-04,성동구,-,강동구 소재 병원,NaN
3,4248,20962,2020-09-04,성북구,-,성북구 사랑제일교회,NaN
4,4247,미부여,2020-09-03,성북구,-,8.15도심집회,NaN


In [85]:
temp = df["확진일"].value_counts().rename_axis('date').reset_index(name='new_confirmed')
temp

,date,new_confirmed
0,2020-08-26,153
1,2020-08-18,151
2,2020-08-15,146
3,2020-08-27,146
4,2020-08-22,140
...,...,...
192,2020-04-15,1
193,2020-04-24,1
194,2020-04-18,1
195,2020-02-2,1


In [86]:
temp["date"]

0      2020-08-26
1      2020-08-18
2      2020-08-15
3      2020-08-27
4      2020-08-22
          ...    
192    2020-04-15
193    2020-04-24
194    2020-04-18
195     2020-02-2
196    2020-04-20
Name: date, Length: 197, dtype: object

In [92]:
temp.set_index("date")

,new_confirmed
date,
2020-08-26,153
2020-08-18,151
2020-08-15,146
2020-08-27,146
2020-08-22,140
...,...
2020-04-15,1
2020-04-24,1
2020-04-18,1


In [93]:
temp.to_csv("./dataset_predict/df_predict.csv", index=False)

In [14]:
temp = pd.read_csv("./dataset_predict/df_predict.csv", encoding="utf-8")

In [18]:
nogada = temp[temp["date"].str.len() < 10]
nogada["date"] = nogada["date"].str.replace("-0", "-")
nogada["date"] = nogada["date"].str.replace("-", "-0")
nogada

,date,new_confirmed
27,2020-03-09,26
37,2020-04-03,20
48,2020-08-07,17
73,2020-08-09,13
79,2020-08-08,13
96,2020-03-08,10
97,2020-08-06,9
99,2020-08-03,9
102,2020-08-05,9
124,2020-03-06,7


In [26]:
index_no = list(nogada.index)
index_no

[27,
 37,
 48,
 73,
 79,
 96,
 97,
 99,
 102,
 124,
 128,
 132,
 140,
 142,
 144,
 151,
 159,
 175,
 182,
 184,
 187,
 188,
 189,
 195]

In [27]:
for i in index_no:
    temp.iloc[i] = nogada.iloc[index_no.index(i)]

In [31]:
df = temp.sort_values(by="date")
df

,date,new_confirmed
181,2020-01-24,1
162,2020-01-30,3
163,2020-01-31,3
195,2020-02-02,1
175,2020-02-05,2
...,...,...
16,2020-08-31,94
13,2020-09-01,101
18,2020-09-02,81
20,2020-09-03,62


In [32]:
df.to_csv("dataset_predict/df_predict.csv", index=False)